In [ ]:
# connecting to database
import sqlite3
db = sqlite3.connect("APAD_proj.db")

# Python Utility Functions

## Function 1: Add a new user

In our application, we are allowing every user to create its own account. Hence, this functionality is not limited only at admin level.

In [ ]:
# Function to check if the user already exists in the database based on the email id
def email_check(db,email):
    cursor = db.cursor()
    while(1):
        findEmail = '''SELECT * FROM user where email_id = ?'''
        cursor.execute(findEmail, (email,))
        if cursor.fetchall():
            print("Email already registered, enter again")
            email = str(input("Please provide your email-address: "))
        else:
            return email

# Function to verify new user's password in order to create account
def password_verify(pwd):
    while pwd[0]!=pwd[1]:
        print("Your passwords didn't match, enter again")
        pwd[0] = str(input("Enter the password: ")) # NEED to add constraints
        pwd[1] = str(input("Enter the password again to recheck: "))
    return pwd[0], pwd[1]
        

#Function to create a new user
def new_user(db, inputs):
    cursor = db.cursor()
    query = '''INSERT INTO user(email_id,first_name,last_name,contact_number, zip_code, password)
                    VALUES(?,?,?,?,?,?)'''
    cursor.execute(query, (inputs[:6]))  
    db.commit()
    print("User added successfully")


# Function to taking inputs from user to create a new account and make a list of all the entries. 
# Output from this function is fed to new_user(db, inputs)
def user_input_form():
    email = str(input("Please provide your email-address: "))
    email = email_check(db, email)
    first_name = str(input("Please Enter your first name: "))
    last_name = str(input("Please Enter your last name: "))
    contact_number = str(input("Please provide a valid contact number: "))
    zip_code = str(input("Your zip code: "))
    password = str(input("Enter the password: ")) # NEED to add constraints
    password_check = str(input("Enter the password again to recheck: "))
    pwd = [password, password_check]
    password, password_check = password_verify(pwd)
    inputs = [email, first_name, last_name, contact_number, zip_code, password, password_check]
    
    return inputs

In [ ]:
# Triggering the add a user functionality
inputs = user_input_form()
new_user(db, inputs)

## Function 2: Add a new venue (admin only)
Only admin can add a new venue. If you will not provide the right admin email id, our system will notify that it is not the admin id and will ask for the correct admin email id.

In [3]:
# Function to check if the entered email is admin's email or not. This function is called where only admin is allowed to make changes

def admin_check(db,email):
    cursor = db.cursor()
    findEmail = '''SELECT email_id FROM user where first_name = ?'''
    cursor.execute(findEmail, ("admin",))
    admin = cursor.fetchone()[0]
    # print(admin)
    while email != admin:
        print("Only admin is authorized to add a venue")
        email = str(input("Enter the admin's email address: "))
    
    return email

# Function to add a new venue in venue table
def new_venue(db, inputs):
    cursor = db.cursor()
    query = '''INSERT INTO venue(venue_name, address, zip_code, contact_number, description, open_time, close_time, games_total_count, games_available_count)
                VALUES(?,?,?,?,?,?,?,?,?)'''
    cursor.execute(query, inputs_venue[1:]) 
    db.commit()

    
# Function to take inputs from the admin to create a venue
# It checks for the admin email id and if entered wrong it will notify that it is not the admin id and will ask for the correct admin email id.
def add_venue_form():
    email = str(input("Please enter your email address: "))
    email1 = admin_check(db,email)
    venue_name = str(input("Please Enter the venue name: "))
    address = str(input("Please Enter the venue address: "))
    zip_code = str(input("Venue zip code: "))
    contact_number = str(input("Please provide a contact number: "))
    description = str(input("Enter the venue description (not more than 200 characters): ")) # Need to limit the length of input
    open_time = str(input("Please Enter the open time in hrs: "))
    close_time   = str(input("Please Enter the close time in hrs: "))
    games_total_count = int(input("Please enter the total number of games that can be played in single time slot: "))
    # games_available_count = int(input("Please enter the available number of games that can be played in single time slot: "))
    games_available_count = 0
    inputs_venue = [email1,venue_name,address,zip_code,contact_number,description,open_time,close_time,games_total_count, games_available_count]
    return inputs_venue

In [4]:
# Triggering the add a venue functionality
inputs_venue = add_venue_form()
new_venue(db, inputs_venue)

Please enter your email address: admin@gmail.com
Please Enter the venue name: zilker park
Please Enter the venue address: research blvd
Venue zip code: 56981
Please provide a contact number: 5602387645
Enter the venue description (not more than 200 characters): multiple games at zilker park
Please Enter the start time in hrs: 7
Please Enter the end time in hrs: 18
Please enter the total number of games that can be played in single time slot: 5


## Function 3: Start an event (user or admin on behalf of user)
This first asks for the user email id and if the user is not registered and ask to create an account first and takes the user to function 1 i.e. add a new user and then brings back to this function.

In [6]:
# Function to verify if the user is registered already or not when he tries to create an event
def email_verify(db, email):
    cursor = db.cursor()
    
    findEmail = '''SELECT email_id FROM user where email_id = ?'''
    cursor.execute(findEmail, (email,))
    
    if cursor.fetchall():
        return email
    else:
        print("You can't create an event yet. Create an account first")
        inputs = user_input_form()
        new_user(db, inputs)
        return inputs[0]

# Function to show the event categories to choose from - till now we have 3 categories: Play/Watch/Workshop    
def fetch_event_cat(cursor):
    event_cat_query = '''SELECT event_category_name FROM event_category'''
    cursor.execute(event_cat_query)
    event_cat = cursor.fetchall()
    print("Please select a category from the following: ") 
    for i in range(len(event_cat)):
        print(str(i) +":" + event_cat[i])
    
    
# Function to fetch all the venues to choose from and display it to the user before taking the input
def fetch_venue(cursor):
    venue_query = '''SELECT venue_name FROM venue'''
    cursor.execute(venue_query)
    venues = cursor.fetchall()
    print("Please select a venue from the following: ") 
    for i in range(len(venues)):
        print(str(i) +":" + venues[i])
    return venues

# Function to fetch all the sports to choose from and display it to the user before taking the input
def fetch_sport(cursor):
    sport_query = '''SELECT sport_name FROM sports_cat'''
    cursor.execute(sport_query)
    sports = cursor.fetchall()
    print("Please select a sport from the following: ") 
    for i in range(len(sports)):
        print(str(i) +":" + sports[i])
    return sports



In [7]:
# Function to create a new event using the outputs of event_create_inputs(db) function
def new_event(db,inputs):
    
    cursor = db.cursor()

    if inputs[9] == "create":

        host_insert_flag = '''INSERT INTO events(event_cat_id,venue_id,event_name, date, start_time,end_time,
                                              user_id, host_flag, member_flag, sports_cat_id, event_desc, capacity_avail)
                       VALUES(?,?,?,?,?,?,?,?,?,?,?,?)'''


        cursor.execute(host_insert_flag, (inputs[7],inputs[8],inputs[1],inputs[2],inputs[3],inputs[4],inputs[0],1,0,inputs[10],inputs[5], inputs[6]))  

        db.commit()
        
        # -1 the count of games_available count in venue table
        
        update_game_avail_ct  = '''UPDATE venue SET games_available_count = ? WHERE venue_id = ?'''
        cursor.execute(update_game_avail_ct,(inputs[11]-1,inputs[8]))
        db.commit()


    elif inputs[9] == "join":

        find_event = '''SELECT * FROM events where date = ?
                            and start_time = ? and end_time = ?'''
        cursor.execute(find_event, (inputs[2], inputs[3], inputs[4],))
        p = cursor.fetchall()
    else:
        print("enter a valid option")

In [8]:
# Function to take inputs from the user or admin to create a new event
def event_create_inputs(db):
    
    count = 0
    cursor = db.cursor()
    db.row_factory = lambda cursor, row: row[0]
    #Input the email of the user
    email = str(input("Please provide your email-address: "))

    #Check if it's an existing user
    email = email_verify(db,email)
    
    create_or_join = str(input("Do you want to create/join the game: ")).lower()
    
    #Get the user_id from the database
    user_id_query = '''SELECT user_id FROM user where email_id = ?'''
    cursor.execute(user_id_query, (email,))
    user_id = cursor.fetchone()
    
    event_name = str(input("Please Enter the event name: "))
    
    #Fetch the list of sports 
    sports = fetch_sport(cursor)
    
    #Input for sport. Only accept if it exists!
    while(1):
        sport_name = str(input("Please Enter the sport name: "))
        for i in sports:
            if (sport_name == i):
                count=1
                break
        if count==1:
            break
        else:
            print("Please enter the correct sport from choices above!")

    #Get the sport id from the database
    sports_cat_id_query = '''SELECT sports_cat_id FROM sports_cat where sport_name = ?'''
    cursor.execute(sports_cat_id_query, (sport_name,))
    sports_cat_id = cursor.fetchone()
    
    event_date = str(input("Please Enter the event date: "))
    start_time = str(input("Please Enter the start time in hrs: "))
    end_time   = str(input("Please Enter the end time in hrs: "))
    event_desc = str(input("Please Enter the event description: "))
    capacity   = str(input("Please Enter the capacity avail: "))
    
    #Fetch all event categories
    fetch_event_cat(cursor)
    
    #Input for event category. Only accept if it exists!
    while(1):
        event_category_name = str(input("Please Enter the event category name : "))
        if (event_category_name == "watch" or event_category_name == "play" or event_category_name == "workshop" ):
            break
        else:
            print("Please enter the right category from choices above!")
    
    #Get the Event cat id from the database
    event_cat_id_query = '''SELECT event_cat_id FROM event_category where event_category_name = ?'''
    cursor.execute(event_cat_id_query, (event_category_name,))
    event_cat_id = cursor.fetchone()
    
    #Fetch all the venues
    venues = fetch_venue(cursor)
    
    #Input for venue. Only accept if it exists!
    count=0
    while(1):
        venue_name = str(input("Please Enter the venue name: "))
        for i in venues:
            if (venue_name == i):
                count=1
                break
        if count==1:
            break
        else:
            print("Please enter the correct venue from choices above!")
    
    #Get the venue id from the database
    venue_id_query = '''SELECT venue_id FROM venue where venue_name = ?'''
    cursor.execute(venue_id_query, (venue_name,))
    venue_id = cursor.fetchone()
    
    #Get the available no. of game counts from a venue
    venue_count_query = '''SELECT games_available_count FROM venue where venue_id = ?'''
    cursor.execute(venue_count_query, (venue_id,))
    games_avail_count = cursor.fetchone()
    
    inputs = [user_id,event_name,event_date,start_time,end_time,event_desc,capacity,event_cat_id,venue_id,create_or_join,sports_cat_id, games_avail_count]
    
    return inputs

In [9]:
cursor = db.cursor()
db.row_factory = lambda cursor, row: row[0]

# Triggering the create an event functionality
inputs2 = event_create_inputs(db)
if inputs2:
    new_event(db,inputs2)

Please provide your email-address: ritika.munjal29@gmail.com
Do you want to create/join the game: create
Please Enter the event name: tennis cup
Please select a sport from the following: 
0:cricket
1:tennis
Please Enter the sport name: tennis
Please Enter the event date: 2019-11-11
Please Enter the start time in hrs: 12
Please Enter the end time in hrs: 14
Please Enter the event description: tennis match
Please Enter the capacity avail: 4
Please select a category from the following: 
0:play
1:watch
2:workshop
Please Enter the event category name : play
Please select a venue from the following: 
0:camden lamar
1:downtown
2:zilker park
Please Enter the venue name: downtown


## Function 4: Display timeslot availability at a venue
Here we have considered that there can be simultaneous events at a venue and that is why we have asked the admin to enter the capacity of the venue to host multiple events at a given time stamp. The output of this function displays the available timeslots and their instances left for a given date and time.

In [10]:
import datetime
from datetime import timedelta

In [11]:
cursor=db.cursor()

In [12]:
# Function to fetch the open and close timings of a venue
def fetch_venue_time(cursor, venue):  
    venue_avail_query = '''SELECT open_time, close_time FROM venue where venue_name = ?'''
    cursor.execute(venue_avail_query, (venue,) )
    venues = cursor.fetchall()
    return venues

# Function to fetch the start time and end time of an event
def fetch_event_time(cursor):
    event_avail_query = '''SELECT start_time, end_time FROM venue, events where venue.venue_id = events.venue_id'''
    cursor.execute(event_avail_query)
    events = cursor.fetchall()
    return events

# Function to fetch the counts of instances of event that can be hosted simultaneously at a venue at a given timestamp
def fetch_avail_slot(cursor, venue):
    venue_avail_query = '''SELECT games_total_count, games_available_count FROM venue where venue_name = ?'''
    cursor.execute(venue_avail_query, (venue,) )
    count = cursor.fetchone()
    return count

# Function to count instances of a tuple in a list and then store it in a dictionary
def count(tuplelist): 
    dct = {} 
    for i in tuplelist: 
        dct[i] = dct.get(i, 0) + 1
    return dct

In [13]:
# Function to find the available timeslots at a venue for a given date
def find_booked_slots(db,venue,date):
    cursor = db.cursor()
    venues = fetch_venue_time(cursor, venue)
    print(venues)
    games_avail_count = fetch_avail_slot(cursor, venue)

    booked_slots = []
    event_date = datetime.datetime.strptime(date, '%Y-%m-%d').date()
    open_time = str(venues[0][0])
    close_time = str(venues[0][1])
    open_time = datetime.datetime.strptime(open_time, '%H').time()
    close_time = datetime.datetime.strptime(close_time, '%H').time()

    event_times = fetch_event_time(cursor)
    print(event_times)
    for i in event_times:
        start_time = str(i[0]) 
        end_time = str(i[1])

        start_time = datetime.datetime.strptime(start_time, '%H').time()
        end_time = datetime.datetime.strptime(end_time, '%H').time()
        #  print(start_time, end_time)
        #  print("Hey")
        #  booked_slots= [((datetime.datetime.combine(event_date,start_time)), (datetime.datetime.combine(event_date,end_time)))]
        #  print(booked_slots)
        #  print(datetime.datetime.combine(event_date,start_time).date()
        booked_slots.append(((datetime.datetime.combine(event_date,start_time)), (datetime.datetime.combine(event_date,end_time))))
    total_hours = ((datetime.datetime.combine(event_date,open_time)), (datetime.datetime.combine(event_date,close_time)))

    duration=timedelta(hours=1)
    slots = sorted([(total_hours[0],total_hours[0])]+ booked_slots + [(total_hours[1],total_hours[1])])
    print("-----Time Slots Avalibility at " + venue + "----\n")
    for start, end in ((slots[i][1], slots[i+1][0]) for i in range(len(slots)-1)):
    #     assert start <= end, "Cannot attend all appointments"
        while start + duration <= end:
            if games_avail_count!=0:
                print ("{:%H:%M} - {:%H:%M}".format(start, start + duration) + "\t" + "(" + str(games_avail_count[0]) + ")")
                start += duration
    hours = []    
    for time,end in booked_slots:   

        while time <= end:
            hours.append(time)
            time += datetime.timedelta(hours=1)

    hours = (sorted(hours))    
#     print(hours)
    temp = []
    i=0
    while(i < len(hours)-1):
        temp.append((hours[i],hours[i]+timedelta(hours=1)))
        i+=1
#     print(temp)
    temp1 = count(temp)
    
    end_time_list = [j for i,j in event_times]
#     print(temp1)
    for i in end_time_list:
        for j in temp1:
            if j[0].hour == i: 
                temp1[j] = temp1[j]-1
#     print(temp1)
    for items in temp1:
        
        print("{:%H:%M} - {:%H:%M}".format(items[0],items[1]) + "\t" + "(" + str(games_avail_count[0]-temp1[items]) + ")")    

In [ ]:
date = "2019-07-27"
venue = "camden lamar"
find_booked_slots(db,venue,date)

# Function 5 : Display all venues where a particular timeslot is available
 #### Here we are displaying all venue names that are available in time slot provided by the user

In [14]:
cursor = db.cursor()
def display_venues_given_time(cursor):

    event_date = str(input("Please Enter the event date in yyyy-mm-dd format: "))
    start_time = str(input("Please Enter the start time in hrs: "))
    end_time   = str(input("Please Enter the end time in hrs: "))
    
    
    display_venue = ''' select venue_name from events e, venue v 
                        where e.venue_id = v.venue_id
                        and e.date = ? and e.start_time = ? and e.end_time = ?
                        and v.games_available_count > 0
                        and e.start_time > v.open_time and e.start_time < v.close_time
                        and e.end_time > e.start_time and e.end_time > v.open_time and e.end_time < v.close_time'''
    
    cursor.execute(display_venue, (event_date,start_time,end_time,) )
    if cursor.fetchall():
        venues = cursor.fetchall()
        print("Available venues at given time slot are: ",venues[0])
    else:
        print("No slots available")

In [15]:
display_venues_given_time(cursor)

Please Enter the event date in yyyy-mm-dd format: 2019-11-11
Please Enter the start time in hrs: 12
Please Enter the end time in hrs: 14
No slots available


# Function 5 : Display all events for a particular venue for given date and time
#### Here we are displaying all event names of venue, date and time selected by the user

In [24]:
cursor = db.cursor()
db.row_factory = lambda cursor, row: row[0]
def display_events_given_time(cursor):
    
    
    event_date = str(input("Please Enter the event date in yyyy-mm-dd format: "))
    start_time = str(input("Please Enter the start time in hrs: "))
    end_time   = str(input("Please Enter the end time in hrs: "))
    venues = fetch_venue(cursor)
    #Input for venue. Only accept if it exists!
    count=0
    while(1):
        venue_name = str(input("Please Enter the venue name: "))
        for i in venues:
            if (venue_name == i):
                count=1
                break
        if count==1:
            break
        else:
            print("Please enter the correct venue from choices above!")
    
    display_event = ''' select e.event_name 
                        from events e, venue v 
                        where e.venue_id = v.venue_id
                        and e.date = ? and e.start_time = ? and e.end_time = ? and v.venue_name = ?'''
    
    cursor.execute(display_event, (event_date,start_time,end_time,venue_name,) )
    events = cursor.fetchall()
    if events:
        print("Available events on for given time and date are: ",events)
    else:
        print("No events available")

In [25]:
display_events_given_time(cursor)

Please Enter the event date in yyyy-mm-dd format: 2019-11-11
Please Enter the start time in hrs: 12
Please Enter the end time in hrs: 14
Please select a venue from the following: 
0:camden lamar
1:downtown
2:zilker park
Please Enter the venue name: downtown
Available events on for given time and date are:  ['tennis cup']
